In [12]:
## Libraries Required
!pip install langchain-huggingface
## For API Calls
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install  bitsandbytes
!pip install langchain


In [1]:
from google.colab import userdata
sec_key = userdata.get('HF_Token')

**HuggingFaceEndpoint**

How to Access HuggingFace Models with API

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from langchain_huggingface import HuggingFaceEndpoint


In [4]:
import os
os.environ['HF_Token']=sec_key


In [5]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(repo_id=repo_id, max_length=128,temperature= 0.7,token=sec_key)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
llm.invoke("What is the capital of India?")

' The capital of India is New Delhi. New Delhi is a city that combines the ancient and the modern. It is the political and administrative hub of India and is home to many government buildings, including the Rashtrapati Bhavan, the official residence of the President of India, and the Parliament House. The city is also home to many historical and cultural landmarks, such as the Red Fort, the Jama Masjid, and the Qutub Minar. New Delhi is a vibrant and bustling city, with a population of over 20 million people. It is a major tourist destination, with many attractions and activities to offer visitors.'

In [7]:
from langchain import PromptTemplate,LLMChain

question = "Who won the cricket world cup 2011?"
template = """Question: {question}
Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
print(prompt.format(question=question))

In [8]:
llm_chain = LLMChain(llm=llm,prompt=prompt)
print(llm_chain.invoke(question))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


**HUgging Face Pipeline**

In [9]:
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [10]:
model_id="gpt2"
model=AutoModelForCausalLM.from_pretrained(model_id)
tokenizer=AutoTokenizer.from_pretrained(model_id)

In [11]:
pipe=pipeline("text-generation",model=model,tokenizer=tokenizer,max_new_tokens=100)
hf=HuggingFacePipeline(pipeline=pipe)

In [12]:
hf

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7c8f3347fdf0>)

In [17]:
hf.invoke("Who invented Machine Learning?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Who invented Machine Learning? And why did the world eventually get interested in computers? How did that process of "imperfectation" begin? And why were computer vision systems developed (and not just for AI); why did the scientific community, especially the public, begin paying attention to the potential problems of AI and its potential to revolutionize the field?\n\nIt\'s interesting that, if you look at this study, it\'s the first study that specifically addresses the question that many scientists have been asking: whether, if ever'

In [19]:
## Use HuggingfacePipelines With Gpu
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    device_map="auto",  # replace with device_map="auto" to use the accelerate library.
    pipeline_kwargs={"max_new_tokens": 100},
)


In [20]:
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

In [21]:
chain = LLMChain(llm=gpu_llm, prompt=prompt)

In [22]:
chain = prompt | gpu_llm

In [24]:
chain.invoke("Who invented Machine Learning?")

"Question: Who invented Machine Learning?\n\nAnswer: Let's think step by step. After a decade of work, we had a machine learning framework to talk to the customer before they could tell us what they were looking for or their preferences. I did a post-processing experiment where I used the Google Translate tool to generate a set of Google documents for testing. Once I got past the stage where we had a system in place, I found out that the tool was not accurate yet so I tested it to see how well it performed against the other, smaller systems in other networks."